# Arbitrage scanner
You need to write CLI application that finds arbitrage opportunities between 2 Uniswap-like DEXes 
([Uniswap](https://v2.info.uniswap.org/home) and [Sushiswap](https://www.sushi.com/)) either for specific block range 

```bash
# Usage example
# Find all arbitrage opportunities in [from-block, to-block] for pairs.csv
python3 ascanner_range.py \
    --from-block 16000000 \
    --to-block 16000100 \
    --pairs pairs.csv \
    --provider-uri http://localhost:8545
```
or for newly mined blocks:

```bash
# Usage example
# Find all arbitrage opportunities in stream mode
python3 ascanner_stream.py --pairs pairs.csv --provider-uri http://localhost:8545
```
In other words, `ascanner_stream.py` should work forever and check every N seconds whether new block was mined and find 
arbitrage opportunities for predefined number of pairs in newly mined blocks.

## Arbitrage Condition
We firstly suggest you state the problem carefully and do some math. Let us consider the pair of 2 tokens $(x, y)$. 
Arbitrage opportunity exists if one swaps $\delta x$ for $\delta y$ on the first DEX with further swapping of $\delta y$ 
for $\delta x' > \delta x$ on the another DEX. Thus, we need to solve the following optimization problem
$$
    \delta x ' (\delta x) - \delta x \longrightarrow \max_{\delta x},
$$
Note that the equation $ \delta x ' (\delta x) - \delta x $ is simply the representation for your profit in $x$ token after 
the execution of arbitrage opportunity. In other words, we are simply solving quite natural problem
$$
    \operatorname{profit} \longrightarrow \max
$$
Note that if the maximized profit is less than 0 then the arbitrage opportunity doesn't exist.

Let us suppose that we observe reserves of 2 identical $(x, y)$ pairs on the 2 DEXes of Uniswapv2 type:
$$
    (x_1, y_1), \quad (x_2, y_2)
$$
In the equations above $(x_1, y_1)$ are token reserves of $(x,y)$-pair on the first DEX and $(x_2, y_2)$ are the reserves 
of $(x,y)$-pair on the second DEX. Without loss of generality let us suppose that we firstly swap $\delta x$ for $\delta y$ on the first DEX
with further swapping of $\delta y$ for $\delta x '$ on second DEX. Thus, the functional representation for $\delta x'(\delta x) - \delta x$ 
can be derived from the following equations:
$$
    x_1 y_1 = (x_1 + r_1\delta x) (y_1 - \delta y), \qquad
    x_2 y_2 = (x_1 - \delta x ') (y_2 + r_2\delta y),
$$
You just need to derive $\delta y = ...$ from the left equation and substitute into the right one. 
Note that $r_i = 1 - \phi_i$, where $\phi_i$ are the DEXes' fees. For instance, Sushiswap and Uniswap fees are the same and equal $0.3\%$. 

**In your code it is reasonable to calculate fees in the so called BASIS POINTS (BPS). 1 BPS equals to 0.01\% = 0.0001. Thus, 0.3\% = 30 BPS and 1 - 0.3\% = 9970 BPS.**

After deriving the functional representation for $\delta x'(\delta x) - \delta x$ you are ready to solve the maximization problem
$$
    \delta x ' (\delta x) - \delta x \longrightarrow \max_{\delta x},
$$
by writing first order condition (first order derivative equals zero). Do not forget to check whether you derived maximum indeed. 

After solving the maximization problem one can derive the arbitrage condition from the following inequality
$$
\delta x ' (\delta x_{\max}) - \delta x_{\max} > 0,
$$ 
where 
$$
    \delta x_{\max} = \arg \max_{\delta x} \left[\delta x ' (\delta x) - \delta x\right].
$$
Thus, your optimal profit can be calculated via 
$$
    \operatorname{profit}_{\max} = \delta x ' (\delta x_{\max}) - \delta x_{\max} 
$$
Strictly speaking, there is also the gas cost that one need to encounter. However, at the moment it is recommended not to think about it. 
There is plenty of work to do already.

## Test your results
Please, go to `utils/arbitrage.py` and implement all the functions there after you have done the math. Verify your code and math calculations with the instructors.

## Looking for pairs
It's time to specify the pairs that you are going to use for arbitrage opportunities. It is reasonable to look through all the pairs that 
UniswapV2 and SuhiswapV2 have and then construct pivot table of all the common pairs of 2 DEXes.

For simplicity we will consider the arbitrage opportunities where $x$ token is WETH and $y$ is another token (USDT, USDC, SHIBA, etc.). 
Thus, we will calculate our profit in WETH.

We provide helper code for deriving the list of pairs via batch request. The motivation of using batch requests is the optimized consunming of 
GetBlock requests. Moreover, it is simply much faster.

In [1]:
from utils.providers import get_provider_from_uri
from utils import hex_to_dec
from utils.requests import (
    get_request_all_pairs,
    get_request_balanceof,
    get_request_token0,
    get_request_token1,
    get_request_get_reserves,
)
import json
from const import ADDRESSES, UNISWAPV2_FACTORY_ABI, ERC20_ABI, UNISWAPV2_PAIR_ABI
import pandas as pd
from web3 import Web3

# Let us create py-web3 objects for Ethereum node DDOS
PROVIDER_URI = "<SPECIFY YOUR PROVIDER URL HERE>"
BATCH_W3 = get_provider_from_uri(PROVIDER_URI, batch=True)
W3 = Web3(BATCH_W3)

In [2]:
# How many pairs are there in Ethereum Blockchain? Quite big number, isn't it?
UNISWAPV2_FACTORY_CONTRACT = W3.eth.contract(
    address=ADDRESSES.uniswapv2_factory, abi=UNISWAPV2_FACTORY_ABI
)
n_pairs = UNISWAPV2_FACTORY_CONTRACT.functions.allPairsLength().call()
print("TOTAL NUMBER OF PAIRS", n_pairs)

TOTAL NUMBER OF PAIRS 136679


In [3]:
# Now let us extract pair addresses for the first 100 pairs via 1 batch request
block_number = W3.eth.block_number
pair_ids = range(100)
factory_address = ADDRESSES.uniswapv2_factory

batch_request = json.dumps(
    [
        get_request_all_pairs(
            factory_address, pair_id, block_number, request_id=pair_id
        )
        for pair_id in pair_ids
    ]

)

batch_response = BATCH_W3.make_batch_request(batch_request)

pair_addresses = [
    "0x" + response_item["result"][-40:]
    for response_item in batch_response
]

pair_addresses[:4]

['0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc',
 '0x3139ffc91b99aa94da8a2dc13f1fc36f9bdc98ee',
 '0x12ede161c702d1494612d19f05992f43aa6a26fb',
 '0xa478c2975ab1ea89e8196811f51a7b7ade33eb11']

In [4]:
# Extracting token0, token1

token0_request = json.dumps(
    [
        get_request_token0(pair_address, block_number, request_id=i)
        for i, pair_address in enumerate(pair_addresses)
    ]
)

batch_response = BATCH_W3.make_batch_request(token0_request)
tokens0 = [
    "0x" + response_item["result"][-40:]
    for response_item in batch_response
]

token1_request = json.dumps(
    [
        get_request_token1(pair_address, block_number, request_id=i)
        for i, pair_address in enumerate(pair_addresses)
    ]
)
batch_response = BATCH_W3.make_batch_request(token1_request)
tokens1 = [
    "0x" + response_item["result"][-40:]
    for response_item in batch_response
]

In [5]:
# Well, let us calculate how many WETH tokens are located in each pair

liquidity_request = json.dumps(
    [
        get_request_balanceof(ADDRESSES.weth, pair_address, block_number, request_id=i)
        for i, pair_address in enumerate(pair_addresses)
    ]
)
batch_response = BATCH_W3.make_batch_request(liquidity_request)
balances = [
    hex_to_dec(response_item["result"])
    for response_item in batch_response
]
balances[:4]

[33216125603148673256243, 0, 328253926942471445, 5062607703275810841392]

In [6]:
# Finalize our results

pd.DataFrame(
    {
        "pair_address": pair_addresses,
        "token0": tokens0,
        "token1": tokens1,
        "WETH_balance": balances,
    }
)

,pair_address,token0,token1,WETH_balance
0,0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,33216125603148673256243
1,0x3139ffc91b99aa94da8a2dc13f1fc36f9bdc98ee,0x8e870d67f660d95d5be530380d0ec0bd388289e1,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0
2,0x12ede161c702d1494612d19f05992f43aa6a26fb,0x06af07097c9eeb7fd685c692751d5c66db49c215,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,328253926942471445
3,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x6b175474e89094c44da98b954eedeac495271d0f,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,5062607703275810841392
4,0x07f068ca326a469fc1d87d85d448990c8cba7df9,0x408e41876cccdc0f92210600ef50372656052a38,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0
...,...,...,...,...
95,0xfb7a3112c96bbcfe4bbf3e8627b0de6f49e5142a,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xe25b0bba01dc5630312b6a21927e578061a13f55,203705096475888814
96,0xaac52b03898359a9a948dac6027334d75c8c64e9,0x25b63bca43914b7d7ccd59892b762c06493a04e6,0x7a5b6651008108a49e07f4d601215c4374005c90,0
97,0xf1f27db872b7f6e8b873c97f785fe4f9a6c92161,0x0000000000085d4780b73119b644ae5ecd22b376,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0
98,0x167acb2fd872a14187a02fc1281790d1de1b4145,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,0xdd974d5c2e2928dea5f71b9825b8b646686bd200,0


## Creating of the pivot table

Now you are ready to extract all the pairs and create the pivot table. 

The procedure for you to do is the following.

1. Extact all UniswapV2 pairs and their WETH balances. You will need to do several batch requests for the task. It is not possible to derive all the pairs with one batch request. 
2. Do the same for SushiswapV2 pairs. `factory_address = ADDRESSES.sushiswapv2_factory`
3. In both dataframes delete all rows that do not contain WETH address in token0 and token1. FYI, WETH address = `ADDRESSES.weth.lower()`
4. In both dataframes delete all rows which WETH_balance is less than $10 ^ {18}$. 
5. Merge all pairs by token0 and token1. Your final dataframe should have length 150-200.

## getReserves batch request

It is reasonable to use batch requests for extracting reserves from pairs. This can be done via the following code.

In [21]:
get_reserves_request = json.dumps(
    [
        get_request_get_reserves(pair_address, block_number, request_id=i)
        for i, pair_address in enumerate(pair_addresses)
    ]
)
batch_response = BATCH_W3.make_batch_request(get_reserves_request)

batch_response
reserves = balances = [
    (hex_to_dec(response_item["result"][:66]), hex_to_dec(response_item["result"][66:130]))
    for response_item in batch_response
]

reserves[:4]

[(44282443188452, 33216125603148673256243),
 (7317507718931362662, 6193354),
 (375418952104909142033, 328253926942471445),
 (6744084750216587104998975, 5062607703275810841392)]

## Extracting token info

You can extract token info via the following code

In [25]:
token_address = ADDRESSES.weth

token_contract = W3.eth.contract(address=token_address, abi=ERC20_ABI)
{
    "name": token_contract.functions.name().call(),
    "symbol": token_contract.functions.symbol().call(),
    "decimals": token_contract.functions.decimals().call(),
    "total_supply": token_contract.functions.totalSupply().call(),
}

{'name': 'Wrapped Ether',
 'symbol': 'WETH',
 'decimals': 18,
 'total_supply': 3792618247932310393855515}